<a href="https://colab.research.google.com/github/GeneSubra/Capstone-3-Unsupervised-ML/blob/master/HubSpot_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import glob 
import pandas as pd 
import json

path = r'drive/MyDrive/dataset/HubSpot/*'
all_files = glob.glob(path)

li = []
for filename in all_files:
    print(filename)
    try: 
        with open(filename, encoding='utf8') as f:
            df = json.load(f)
            df = pd.json_normalize(df)
        li.append(df)
    except: 
        pass

frame = pd.concat(li, axis=0, ignore_index = True)

drive/MyDrive/dataset/HubSpot/0b7855e4-8c38-4c8d-997f-fb7ffa8c2a3a (1)
drive/MyDrive/dataset/HubSpot/1c7b85d8-79d0-4588-a0b9-3a0dfe6e7a3a (1)
drive/MyDrive/dataset/HubSpot/9cb0d640-8c99-494b-8739-944e5dc32d51 (1)
drive/MyDrive/dataset/HubSpot/40b06154-46f0-4756-9149-3e8a3686d3b6
drive/MyDrive/dataset/HubSpot/343addc5-c728-4498-ad60-97ff9ccd4c4c
drive/MyDrive/dataset/HubSpot/c54fb50e-3280-4149-8149-dd5b63b47a5c (1)
drive/MyDrive/dataset/HubSpot/21112c24-dca4-4103-a700-0327c58f2be3


In [4]:
frame.head()

,First Name,Last Name,LI URL,Email,Phone,Company,Position,Location,Industry,Fit,win,skill0,skill1,skill2,industry0,industry1,industry2,interest0,interest1,interest2,linkedin,email,birth_date,gender,start_date,end_date,title,city,state,country,street,postal,education_name,education_major,education_degree,education_type,education_start,education_end,age,company_name,company_founded,company_industry,company_size,company_website,company_country,company_city,company_state,years_in_business,Contact ID,Lead Status,Contact owner,Create Date,Company Name,Company ID,Name,Phone Number,City,Company owner,Last Activity Date,Country/Region,LinkedIn URL,Became a Customer Date,Became a Lead Date,Associated Company ID,Associated Company,Linkedin URLs,Profile Email,Company Website,STATUS,Win
0,Ashley,Greensmyth,https://linkedin.com/in/ashley-greensmyth,ashley@ashleygreensmyth.com,,Ashley Greensmyth Counselling,Owner,Burnaby British Columbia Canada,Health Wellness & Fitness,No,False,teamwork,payroll,customer service,"health, wellness and fitness",,,social services,civil rights and social action,education,http://www.linkedin.com/in/ashley-greensmyth,,None,female,2018-04,None,owner,burnaby,british columbia,canada,None,None,adler university,counseling,masters,post-secondary institution,2012,2016,29,ashley greensmyth counselling,None,None,None,None,,,,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Earl,Tunheim,https://linkedin.com/in/earl-tunheim-a7a4b33,earl@tunheim.com,,Tunheim Inc. Retired,Retired,Courtenay British Columbia Canada,Computer Software,No,False,sap implementation,sap,sap r/3,computer software,,,,,,http://www.linkedin.com/in/earl-tunheim-a7a4b33,,None,male,,,,courtenay,british columbia,canada,None,None,university of alberta,,bachelors,post-secondary institution,1966,1972,73,,,,,,,,,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KC,Dyer,https://linkedin.com/in/kcdyer1,kc@dyertech.ca,250-402-8647,Swift Internet Services,Lead Network Engineer,Central Kootenay British Columbia Canada,Computer & Network Security,No,False,web development,databases,mysql,computer & network security,,,science and technology,economic empowerment,,http://www.linkedin.com/in/kcdyer1,,None,male,2013-03,None,technical network engineer,creston,british columbia,canada,None,None,prince charles secondary school,computer information systems,,None,1996,2000,45,swift internet services,None,None,None,None,,,,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Margot,Gauley,https://linkedin.com/in/margot-gauley-03bb4348,mgauley@telus.net,,Health and Technology District,Innovation Hub Coordinator,Canada,Events Services,No,False,,,,"health, wellness and fitness",,,,,,http://www.linkedin.com/in/margot-gauley-03bb4348,,None,female,2019-05,None,innovation hub coordinator,None,None,canada,None,None,,,,,,,0,health & technology district,None,None,None,None,,,,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Michael,Nolan,https://linkedin.com/in/michael-nolan-ba0684185,michael@youarecollective.ca,,You Are Collective,Co-Founder,Canada,Public Relations,No,False,sales,office administration,team building,public relations and communications,,,,,,http://www.linkedin.com/in/michael-nolan-ba068...,,None,male,2018-11,None,co-founder,None,None,canada,None,None,,,,,,,0,you are collective,None,None,None,None,,,,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
frame.shape

(423, 70)

In [6]:
frame = frame.drop_duplicates()

In [7]:
frame.shape

(412, 70)

In [8]:
frame['STATUS'].unique()

array([nan, 'Win'], dtype=object)

In [9]:
frame['win'].unique()

array(['False', 'false', 'true', 'True'], dtype=object)

In [10]:
frame['Win'].unique()

array([nan, 'Win'], dtype=object)

In [11]:
features = [f for f in frame.columns if f not in ['win','Win', 'STATUS', 'Lead Status', 'Contact owner',
       'Create Date', 'First Name', 'Last Name', 'LI URL', 'Email', 'Phone','Company Name', 'Company ID', 'Name', 'Phone Number','Contact ID',
       'Company owner', 'Last Activity Date', 'Country/Region', 'Became a Customer Date', 'Became a Lead Date',
       'Associated Company ID', 'Associated Company', 'Linkedin URLs',
       'Profile Email', 'Company Website',
       'LinkedIn URL']]

In [12]:
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [13]:
for feat in features:
  lbl_enc = preprocessing.LabelEncoder()
  frame.loc[:,feat] = lbl_enc.fit_transform(frame[feat].astype(str).fillna("-1"))

In [14]:
frame.columns

Index(['First Name', 'Last Name', 'LI URL', 'Email', 'Phone', 'Company',
       'Position', 'Location', 'Industry', 'Fit', 'win', 'skill0', 'skill1',
       'skill2', 'industry0', 'industry1', 'industry2', 'interest0',
       'interest1', 'interest2', 'linkedin', 'email', 'birth_date', 'gender',
       'start_date', 'end_date', 'title', 'city', 'state', 'country', 'street',
       'postal', 'education_name', 'education_major', 'education_degree',
       'education_type', 'education_start', 'education_end', 'age',
       'company_name', 'company_founded', 'company_industry', 'company_size',
       'company_website', 'company_country', 'company_city', 'company_state',
       'years_in_business', 'Contact ID', 'Lead Status', 'Contact owner',
       'Create Date', 'Company Name', 'Company ID', 'Name', 'Phone Number',
       'City', 'Company owner', 'Last Activity Date', 'Country/Region',
       'LinkedIn URL', 'Became a Customer Date', 'Became a Lead Date',
       'Associated Company ID', 

In [15]:
frame.head()

,First Name,Last Name,LI URL,Email,Phone,Company,Position,Location,Industry,Fit,win,skill0,skill1,skill2,industry0,industry1,industry2,interest0,interest1,interest2,linkedin,email,birth_date,gender,start_date,end_date,title,city,state,country,street,postal,education_name,education_major,education_degree,education_type,education_start,education_end,age,company_name,company_founded,company_industry,company_size,company_website,company_country,company_city,company_state,years_in_business,Contact ID,Lead Status,Contact owner,Create Date,Company Name,Company ID,Name,Phone Number,City,Company owner,Last Activity Date,Country/Region,LinkedIn URL,Became a Customer Date,Became a Lead Date,Associated Company ID,Associated Company,Linkedin URLs,Profile Email,Company Website,STATUS,Win
0,Ashley,Greensmyth,https://linkedin.com/in/ashley-greensmyth,ashley@ashleygreensmyth.com,,13,23,1,11,0,False,123,91,29,29,0,0,51,9,18,14,0,4,1,103,1,129,15,10,1,2,2,2,28,13,2,45,45,4,19,48,1,8,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Earl,Tunheim,https://linkedin.com/in/earl-tunheim-a7a4b33,earl@tunheim.com,,150,26,5,4,0,False,108,116,104,14,0,0,0,0,0,52,0,4,2,0,0,0,26,10,1,2,2,136,0,6,2,3,3,46,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KC,Dyer,https://linkedin.com/in/kcdyer1,kc@dyertech.ca,250-402-8647,143,19,3,3,0,False,132,36,79,13,0,0,48,17,0,116,0,4,2,59,1,155,27,10,1,2,2,94,26,0,1,28,29,20,205,48,1,8,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Margot,Gauley,https://linkedin.com/in/margot-gauley-03bb4348,mgauley@telus.net,,51,18,2,8,0,False,0,0,0,29,0,0,0,0,0,139,0,4,1,116,1,100,1,1,1,2,2,0,0,0,0,0,0,0,78,48,1,8,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Michael,Nolan,https://linkedin.com/in/michael-nolan-ba0684185,michael@youarecollective.ca,,155,5,2,29,0,False,104,88,121,57,0,0,0,0,0,152,0,4,2,109,1,38,1,1,1,2,2,0,0,0,0,0,0,0,229,48,1,8,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def get_model(df, categorical_columns):
  inputs = []
  outputs = []
  for c in categorical_columns:
    num_unique_vals = int(df[c].nunique())
    embed_dim = int(min(np.ceil(num_unique_vals/ 2), 50))
    inp = layers.Input(shape=(1,))
    out = layers.Embedding(num_unique_vals + 1, embed_dim, name = c)(inp)
    # apply dropout here
    out = layers.Reshape(target_shape=(embed_dim, ))(out)
    inputs.append(inp)
    outputs.append(out)
  x = layers.Concatenate()(outputs)
  x = layers.Dense(300, activation='relu')(x)
  x = layers.Dropout(0.3)(x)
  y = layers.Dense(1, activation='sigmoid')(x)
  model = Model(inputs=inputs, outputs = y)
  return model

In [17]:
get_model(frame, features).summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [18]:
model = get_model(frame, features)
model.compile(loss='binary_crossentropy', optimizer='adam')

In [19]:
frame.loc[frame['Win']=='Win', 'target'] = 1
frame.loc[frame['Win']!='Win', 'target'] = 0 

In [20]:
frame['target'].value_counts()

0.0    286
1.0    126
Name: target, dtype: int64

In [21]:
frame['Win'].value_counts()

Win    126
Name: Win, dtype: int64

In [22]:
model.fit([frame.loc[:,f].values for f in features], frame.target.values)

13/13 [==============================] - 6s 10ms/step - loss: 0.6629


In [23]:
def create_model(data, catcols):    
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2), 50))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=y)
    return model

In [24]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [25]:
test_data = [frame.loc[:, features].values[:, k] for k in range(frame.loc[:, features].values.shape[1])]

In [26]:
oof_preds = np.zeros((len(frame)))
test_preds = np.zeros((len(frame)))

skf = StratifiedKFold(n_splits=50)
for train_index, test_index in skf.split(frame, frame.target.values):
    X_train, X_test = frame.iloc[train_index, :], frame.iloc[test_index, :]
    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train, y_test = X_train.target.values, X_test.target.values
    model = create_model(frame, features)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
    X_train = [X_train.loc[:, features].values[:, k] for k in range(X_train.loc[:, features].values.shape[1])]
    X_test = [X_test.loc[:, features].values[:, k] for k in range(X_test.loc[:, features].values.shape[1])]
    
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=5,
                                 verbose=1, mode='max', baseline=None, restore_best_weights=True)

    rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
                                      patience=3, min_lr=1e-6, mode='max', verbose=1)
    
    model.fit(X_train,
              utils.to_categorical(y_train),
              validation_data=(X_test, utils.to_categorical(y_test)),
              verbose=1,
              batch_size=1024,
              callbacks=[es, rlr],
              epochs=100
             )
    valid_fold_preds = model.predict(X_test)[:, 1]
    test_fold_preds = model.predict(test_data)[:, 1]
    oof_preds[test_index] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    print(metrics.roc_auc_score(y_test, valid_fold_preds))
    K.clear_session()

Epoch 1/100
1/1 [==============================] - 9s 9s/step - loss: 0.9346 - auc: 0.4663 - val_loss: 0.6907 - val_auc: 0.4444
Epoch 2/100
1/1 [==============================] - 0s 131ms/step - loss: 0.8305 - auc: 0.6132 - val_loss: 0.6871 - val_auc: 0.5556
Epoch 3/100
1/1 [==============================] - 0s 121ms/step - loss: 0.7577 - auc: 0.7029 - val_loss: 0.6844 - val_auc: 0.5556
Epoch 4/100
1/1 [==============================] - 0s 120ms/step - loss: 0.7468 - auc: 0.7258 - val_loss: 0.6823 - val_auc: 0.5556
Epoch 5/100
1/1 [==============================] - 0s 119ms/step - loss: 0.7218 - auc: 0.7439 - val_loss: 0.6805 - val_auc: 0.5556

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/100
1/1 [==============================] - 0s 127ms/step - loss: 0.7010 - auc: 0.7592 - val_loss: 0.6795 - val_auc: 0.5556
Epoch 7/100
1/1 [==============================] - 0s 132ms/step - loss: 0.7114 - auc: 0.7474 - val_loss: 0.6787 - val_auc: 0.5556
Resto

In [27]:
print("Overall AUC={}".format(metrics.roc_auc_score(frame.target.values, oof_preds)))

Overall AUC=0.5484237984237984


In [28]:
test_preds /= 50
test_ids = frame['Contact ID'].values
print("Saving submission file")
submission = pd.DataFrame.from_dict({
    'id': test_ids,
    'target': test_preds
})
submission.to_csv("drive/MyDrive/dataset/HubSpot/submission.csv", index=False)

Saving submission file


In [29]:
submission[submission['id'] == "429801"]

,id,target
66,429801,0.475934


In [30]:
ids = submission['id'].unique()[1:].tolist()

In [31]:
submission[submission['id'].isin(ids)]

,id,target
38,179201,0.479742
39,6661,0.476987
40,178951,0.478656
41,86151,0.480996
42,464651,0.474576
43,86801,0.477410
44,3601,0.479164
45,147351,0.478132
46,379801,0.478307
47,8864,0.474596


In [32]:
frame[frame['Contact ID'].isin(ids)][['Contact ID', 'target']]

,Contact ID,target
39,179201,0.0
40,6661,0.0
41,178951,0.0
42,86151,0.0
43,464651,0.0
44,86801,0.0
45,3601,0.0
46,147351,0.0
47,379801,0.0
48,8864,0.0


In [33]:
frame[frame['target']==1].head()

,First Name,Last Name,LI URL,Email,Phone,Company,Position,Location,Industry,Fit,win,skill0,skill1,skill2,industry0,industry1,industry2,interest0,interest1,interest2,linkedin,email,birth_date,gender,start_date,end_date,title,city,state,country,street,postal,education_name,education_major,education_degree,education_type,education_start,education_end,age,company_name,company_founded,company_industry,company_size,company_website,company_country,company_city,company_state,years_in_business,Contact ID,Lead Status,Contact owner,Create Date,Company Name,Company ID,Name,Phone Number,City,Company owner,Last Activity Date,Country/Region,LinkedIn URL,Became a Customer Date,Became a Lead Date,Associated Company ID,Associated Company,Linkedin URLs,Profile Email,Company Website,STATUS,Win,target
297,MAUREEN,Greene,NaN,NaN,NaN,80,28,15,32,2,True,0,0,0,47,0,0,0,0,0,148,0,4,1,11,1,129,52,28,3,2,2,0,0,0,0,0,0,0,124,48,48,7,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/in/maureen-greene-556...,maureen@mgpromotions.co.uk,http://mgpromotions.co.uk/,NaN,Win,1.0
298,Pierre,van der Heyden,NaN,NaN,NaN,4,28,15,32,2,True,26,82,47,14,0,0,26,0,0,175,50,4,2,127,1,48,52,34,3,2,2,73,37,13,2,45,43,6,5,45,11,4,6,7,1,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/in/pierrevanderheyden/,pierre.vanderheyden@adarga.ai,http://www.adarga.ai,NaN,Win,1.0
299,Yma,Koomson,NaN,NaN,NaN,6,28,15,32,2,True,82,103,67,58,0,0,25,9,18,253,0,4,1,72,1,111,1,1,3,2,2,4,34,6,1,4,6,43,9,48,46,1,10,7,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/in/ymakoomson/,yma@ahic.org.uk,http://www.ahic.org.uk/,NaN,Win,1.0
300,Hanna,Guy,NaN,NaN,NaN,7,28,15,32,2,True,0,0,0,10,0,0,0,0,0,80,0,4,1,108,1,146,1,1,3,2,2,0,0,0,0,0,0,0,10,40,12,1,1,7,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/in/hanna-guy-5263252a/,hanna@alltimescoatings.co.uk,http://www.alltimescoatings.co.uk/,NaN,Win,1.0
301,George,Pornaris,NaN,NaN,NaN,8,28,15,32,2,True,43,16,113,4,0,0,59,34,35,70,0,4,2,64,1,107,52,34,3,2,2,62,7,0,1,17,15,34,11,48,4,4,11,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/in/george-pornaris-34...,george@alphabetpie.com,http://www.alphabetpie.com/,NaN,Win,1.0


In [34]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')